In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("venky73/spam-mails-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/venky73/spam-mails-dataset/versions/1


In [4]:
import string
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
df = pd.read_csv(path + '/spam_ham_dataset.csv')
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [7]:
df['text']=df['text'].apply(lambda x:x.replace('\r\n',''))

In [10]:
df.text.iloc[2]

"Subject: neon retreatho ho ho , we ' re around to that most wonderful time of the year - - - neon leaders retreat time !i know that this time of year is extremely hectic , and that it ' s tough to think about anything past the holidays , but life does go on past the week of december 25 through january 1 , and that ' s what i ' d like you to think about for a minute .on the calender that i handed out at the beginning of the fall semester , the retreat was scheduled for the weekend of january 5 - 6 . but because of a youth ministers conference that brad and dustin are connected with that week , we ' re going to change the date to the following weekend , january 12 - 13 . now comes the part you need to think about .i think we all agree that it ' s important for us to get together and have some time to recharge our batteries before we get to far into the spring semester , but it can be a lot of trouble and difficult for us to get away without kids , etc . so , brad came up with a potentia

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5171 non-null   int64 
 1   label       5171 non-null   object
 2   text        5171 non-null   object
 3   label_num   5171 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.7+ KB


In [15]:
stemmer = PorterStemmer()
corpus = []

stopwords_set = set(stopwords.words('english'))

for i in range(len(df)):
  text = df['text'].iloc[i].lower()
  text = text.translate(str.maketrans('','',string.punctuation)).split()
  text = [stemmer.stem(word) for word in text if word not in stopwords_set]
  text = ' '.join(text)
  corpus.append(text)

In [16]:
corpus[0]

'subject enron methanol meter 988291thi follow note gave monday 4 3 00 preliminaryflow data provid daren pleas overrid pop daili volum present zero reflect dailyact obtain ga control chang need asap econom purpos'

In [17]:
df.text.iloc[0]

"Subject: enron methanol ; meter # : 988291this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminaryflow data provided by daren } .please override pop ' s daily volume { presently zero } to reflect dailyactivity you can obtain from gas control .this change is needed asap for economics purposes ."

In [19]:
from tkinter.constants import X
vectorizer = CountVectorizer()

x = vectorizer.fit_transform(corpus).toarray()
y = df['label_num'].values

X_train , X_test , y_train , y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [21]:
x[0]

array([1, 0, 0, ..., 0, 0, 0])

In [22]:
clf = RandomForestClassifier(n_jobs=1)
clf.fit(X_train,y_train)

RandomForestClassifier(n_jobs=1)

In [23]:
clf.score(X_test,y_test)

0.9739130434782609

In [24]:
email_to_classify = df.text.values[10]

In [25]:
email_to_classify

"Subject: vocable % rnd - word asceticismvcsc - brand new stock for your attentionvocalscape inc - the stock symbol is : vcscvcsc will be our top stock pick for the month of april - stock expected tobounce to 12 cents levelthe stock hit its all time low and will bounce backstock is going to explode in next 5 days - watch it soarwatch the stock go crazy this and next week .breaking news - vocalscape inc . announces agreement to resell mix networkservicescurrent price : $ 0 . 025we expect projected speculative price in next 5 days : $ 0 . 12we expect projected speculative price in next 15 days : $ 0 . 15vocalscape networks inc . is building a company that ' s revolutionizing thetelecommunications industry with the most affordable phone systems , hardware ,online software , and rates in canada and the us . vocalscape , a company withglobal reach , is receiving international attention for the development of voiceover ip ( voip ) application solutions , including the award - winning eyefont

In [26]:
email_text = email_to_classify.lower().translate(str.maketrans('','',string.punctuation)).split()
email_text = [stemmer.stem(word) for word in email_text if word not in stopwords_set]
email_text = ' '.join(email_text)

email_corpus = [email_text]

In [27]:
x_email = vectorizer.transform(email_corpus)

In [29]:
clf.predict(x_email)

array([1])

In [30]:
df.label_num.iloc[15]

1